In [7]:
import numpy as np
import pandas as pd

df = pd.read_csv('smoke_detector_task.csv')
df = df.drop(columns = ['Unnamed: 0'])
columns_to_repair = ['Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2', 'Pressure[hPa]', 'PM2.5', 'NC1.0']
for i in columns_to_repair:
    df.loc[df[i].isna(), i] = df[i].mean()
df

,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,1654733331,20.000,57.36,0.0,400.0,12306.0,18520,939.735,0.00,0.00,0.00,0.000,0.000,0,No
1,1654733332,20.015,56.67,0.0,400.0,12345.0,18651,939.744,0.00,0.00,0.00,0.000,0.000,1,No
2,1654733333,20.029,55.96,0.0,400.0,12374.0,18764,939.738,0.00,0.00,0.00,0.000,0.000,2,No
3,1654733334,20.044,55.28,0.0,400.0,12390.0,18849,939.736,0.00,0.00,0.00,0.000,0.000,3,No
4,1654733335,20.059,54.69,0.0,400.0,12403.0,18921,939.744,0.00,0.00,0.00,0.000,0.000,4,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,1655130047,18.438,15.79,625.0,400.0,13723.0,20569,936.670,0.63,0.65,4.32,0.673,0.015,5739,No
62626,1655130048,18.653,15.87,612.0,400.0,13731.0,20588,936.678,0.61,0.63,4.18,0.652,0.015,5740,No
62627,1655130049,18.867,15.84,627.0,400.0,13725.0,20582,936.687,0.57,0.60,3.95,0.617,0.014,5741,No
62628,1655130050,19.083,16.04,638.0,400.0,13712.0,20566,936.680,0.57,0.59,3.92,0.611,0.014,5742,No


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
X = df.drop(columns = ['Fire Alarm'])
y = df['Fire Alarm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)



In [19]:
prediction = X_train - X_test.iloc[0]

KeyError: 0

In [45]:
k = 5

In [28]:
predictions = []
    for i in range(len(X_test)):
        distances = []
        labels = {}

        for q in range(len(self.X_train)):
            distance = np.sqrt(np.sum(np.square(self.X_train - X_test.iloc[i])))
            distances.append([distance, self.y_train[q]])

        distances.sort(key = lambda x: x[0])
        distances = distances[:self.k]
        for label in distances:
            if label in labels:
                labels[label] += 1
            else:
                labels[label] = 1
        predictions.append(max(labels, key = labels.get))
return (predictions)

UTC               1.654733e+09
Temperature[C]    1.900000e+01
Humidity[%]       5.636000e+01
TVOC[ppb]        -1.000000e+00
eCO2[ppm]         3.990000e+02
Raw H2            1.230500e+04
Raw Ethanol       1.851900e+04
Pressure[hPa]     9.387350e+02
PM1.0            -1.000000e+00
PM2.5            -1.000000e+00
NC0.5            -1.000000e+00
NC1.0            -1.000000e+00
NC2.5            -1.000000e+00
CNT              -1.000000e+00
Name: 0, dtype: float64


In [56]:
labels = {'1': 4}
a = {"1": 2, "3": 4}
for label in a:
    if label in labels:
        labels[label] += 1
    else:
        labels[label] = 1
print(labels)

{'1': 5, '3': 1}


In [58]:
max(labels, key = labels.get)

'1'